# University Student Dropout: A Longitudinal Dataset of Demographic, Socioeconomic, and Academic Indicators
In this document I walk through the data provided by this research and do basic training before separating code into final executables


In [10]:
import pandas as pd
import re
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

## Loading datasets for 2021-2022
The goal is to compare them and merge into single dataset for 2 years
I decided to focus on two most recent datasets

The datasets have relative naming for months like 'pft_events_2021_10'
Rename them to same format and merge

In [3]:
df_2021 = pd.read_csv("../data/raw/dataset_2021_hash.csv", sep=';', low_memory=False, decimal=',')
df_2022 = pd.read_csv("../data/raw/dataset_2022_hash.csv", sep=';', low_memory=False, decimal=',')

In [4]:
print(f"Shapes: 2021:{df_2021.shape}; 2022:{df_2022.shape}")
print(f"Columns: 2021: {len(df_2021.columns)}; 2022:{len(df_2022.columns)}")

cols_2021 = set(df_2021.columns)
cols_2022 = set(df_2022.columns)

print(f"Columns in 2021 but not 2022: {cols_2021 - cols_2022}")
print(f"Columns in 2022 but not 2021: {cols_2022 - cols_2021}")
print(f"Shared columns: {len(cols_2021.intersection(cols_2022))}")

Shapes: 2021:(153120, 178); 2022:(159173, 169)
Columns: 2021: 178; 2022:169
Columns in 2021 but not 2022: {'pft_days_logged_2022_6', 'n_resource_days_2022_2', 'n_resource_days_2021_9', 'resource_events_2021_12', 'pft_days_logged_2021_9', 'pft_assignment_submissions_2021_12', 'n_resource_days_2022_1', 'n_wifi_days_2021_9', 'pft_assignment_submissions_2021_11', 'pft_visits_2021_10', 'resource_events_2021_9', 'n_wifi_days_2022_2', 'pft_total_minutes_2021_10', 'pft_total_minutes_2022_7', 'pft_events_2021_10', 'pft_events_2021_12', 'n_resource_days_2022_7', 'pft_events_2021_9', 'pft_visits_2022_1', 'pft_total_minutes_2022_8', 'pft_assignment_submissions_2022_6', 'pft_assignment_submissions_2022_3', 'pft_visits_2022_5', 'pft_assignment_submissions_2022_5', 'pft_days_logged_2022_4', 'n_resource_days_2022_5', 'pft_test_submissions_2022_4', 'n_wifi_days_2022_4', 'pft_assignment_submissions_2022_2', 'pft_test_submissions_2022_2', 'n_resource_days_2021_12', 'resource_events_2022_8', 'pft_test_sub

In [5]:
def align_academic_months(df, cohort_year):
    """
    Renames columns like 'pft_events_2021_10' to 'pft_events_m2' 
    assuming September is Month 1.
    """
    new_cols = {}
    for col in df.columns:
        # Match pattern like _2021_9 or _2022_1
        match = re.search(r'_(\d{4})_(\d{1,2})$', col)
        if match:
            year = int(match.group(1))
            month = int(match.group(2))
            
            # Relative month (Sept=1, Oct=2... Aug=12)
            rel_month = (month - 9) % 12 + 1
            
            base_name = col[:match.start()]
            new_cols[col] = f"{base_name}_m{rel_month}"
            
    return df.rename(columns=new_cols)


In [6]:
df_2021 = align_academic_months(df_2021, 2021)
df_2022 = align_academic_months(df_2022, 2022)

print(f"Shapes: 2021:{df_2021.shape}; 2022:{df_2022.shape}")
print(f"Columns: 2021: {len(df_2021.columns)}; 2022:{len(df_2022.columns)}")

# 2021 set has 9 more columns which we drop by taking intersection:
shared_cols = list(set(df_2021.columns).intersection(set(df_2022.columns)))

df_2021 = df_2021[shared_cols]
df_2022 = df_2022[shared_cols]



cols_2021 = set(df_2021.columns)
cols_2022 = set(df_2022.columns)

print(f"Shared columns: {len(cols_2021.intersection(cols_2022))}")



Shapes: 2021:(153120, 178); 2022:(159173, 169)
Columns: 2021: 178; 2022:169
Shared columns: 169


### Merge both datasets now

In [18]:

# Cohort year indicator
df_2021['cohort_year'] = 2021
df_2022['cohort_year'] = 2022

df_combined = pd.concat([df_2021, df_2022], ignore_index=True)
print(f"Combined shape: {df_combined.shape}")
print(df_combined['abandono_hash'].value_counts())
# Seems that 'B' means not abandoned, 'A' means abandoned
df_combined['target'] = df_combined['abandono_hash'].map({'B': 0, 'A': 1})
df_combined.drop(columns=['abandono_hash'], inplace=True)

Combined shape: (312293, 170)
abandono_hash
B    292561
A     19732
Name: count, dtype: int64


## Data analysis and cleaning

Examining data and preparing it for use in XGBoost

In [19]:
print(f"Shape: {df_combined.shape}")

# print(f"\nHow many missing values per column, given {df_combined.shape[0]} rows")
# print(df_2022.isnull().sum().to_string())



missing_percent = (df_combined.isnull().sum() / len(df_combined)) * 100
print("Percentage of missing values per column:")
print(missing_percent.sort_values(ascending=False).to_string())


print(df_combined.describe().to_string())

df_combined.head()




Shape: (312293, 170)
Percentage of missing values per column:
pft_test_submissions_m11          99.978546
pft_assignment_submissions_m11    99.837973
es_retitulado                     99.585325
total1                            99.180257
es_adaptado                       98.608999
ajuste                            97.629150
impagado_curso_mat                97.592325
resource_events_m11               96.003433
n_resource_days_m11               96.003433
pft_test_submissions_m5           95.493015
pft_test_submissions_m10          95.435376
pft_test_submissions_m1           95.314336
pft_assignment_submissions_m1     94.707854
pft_assignment_submissions_m5     94.604106
pft_assignment_submissions_m10    94.587775
activ1                            94.542945
pft_test_submissions_m8           94.358183
pft_test_submissions_m6           94.304387
pft_visits_m11                    94.255075
pft_total_minutes_m11             94.243547
pft_assignment_submissions_m6     93.635784
pft_test_submi

,rendimiento_cuat_a,pft_days_logged_m4,n_resource_days_m5,pft_days_logged_m9,pft_test_submissions_m7,nota10_hash,nota_asig_hash,rendimiento_total,resource_events_m8,cred_mat5,pft_total_minutes_m3,pft_test_submissions_m9,tipo_ingreso,pft_assignment_submissions_m8,pft_test_submissions_m5,cred_sup_tit,asig1,pft_total_minutes_m9,pft_total_minutes_m10,rendimiento_cuat_b,pft_events_m5,pft_days_logged_m1,pft_days_logged_m5,cred_mat2,pft_events_m1,pft_total_minutes_m7,n_wifi_days_m6,pft_assignment_submissions_m4,pft_events_m6,pft_visits_m1,pft_total_minutes_m6,pft_days_logged_m11,pft_test_submissions_m3,cred_mat_sem_b,pft_days_logged_m8,pft_visits_m10,pft_events_m11,cred_pend_sup_tit,cred_sup_total,asi_hash,pft_assignment_submissions_m9,pft_test_submissions_m4,tit_hash,estudios_m_hash,n_wifi_days_m4,cred_mat_movilidad,pft_days_logged_m6,pft_events_m3,pft_visits_m2,cred_mat_total,cred_ptes_acta,pft_events_m8,pft_visits_m3,pft_test_submissions_m1,dedicacion,cred_sup_4o,impagado_curso_mat,cred_sup_3o,n_wifi_days_m2,es_adaptado,pft_total_minutes_m8,pft_test_submissions_m11,n_wifi_days_m1,n_resource_days_m8,pft_visits_m6,n_wifi_days_m5,cred_sup_normal,pft_events_m2,resource_events_m7,resource_events_m5,pft_assignment_submissions_m11,desplazado_hash,rend_total_ultimo,n_wifi_days_m7,es_retitulado,baja_fecha,ajuste1,pft_total_minutes_m1,cred_mat_anu,activ1,cred_sup_sem_a,pft_total_minutes_m4,cred_sup_1o,cred_mat1,pft_total_minutes_m2,resource_events_m6,cred_sup_6o,n_wifi_days_m9,fecha_datos,cred_mat4,pft_assignment_submissions_m6,cred_sup_espec,cred_mat_sem_a,anyo_inicio_estudios,pft_assignment_submissions_m2,exento_npp,pft_visits_m5,pft_assignment_submissions_m3,pft_test_submissions_m2,curso_mas_alto,pft_days_logged_m7,resource_events_m3,n_resource_days_m10,pft_events_m4,pft_total_minutes_m5,cred_sup,pft_visits_m8,pft_visits_m4,pract1,cred_sup_sem_b,pft_visits_m11,cred_mat_practicas,ajuste,pft_assignment_submissions_m1,pft_visits_m7,cred_sup_anu,n_resource_days_m6,cred_sup_5o,resource_events_m9,pft_days_logged_m2,pft_total_minutes_m11,cred_mat3,n_wifi_days_m8,pft_events_m9,resource_events_m11,pft_events_m10,pft_assignment_submissions_m10,pft_assignment_submissions_m5,total1,resource_events_m4,n_resource_days_m11,curso_mas_bajo,n_resource_days_m9,n_resource_days_m4,rend_total_penultimo,cred_mat_normal,n_resource_days_m7,cred_sup_2o,dni_hash,n_wifi_days_m11,campus_hash,pft_test_submissions_m6,pft_days_logged_m10,resource_events_m10,pft_visits_m9,estudios_p_hash,resource_events_m2,n_resource_days_m3,matricula_activa,n_wifi_days_m10,actividades,practicas,n_resource_days_m2,caca,nota14_hash,pft_test_submissions_m10,pft_test_submissions_m8,pft_assignment_submissions_m7,cred_mat6,resource_events_m1,n_wifi_days_m3,pft_days_logged_m3,rend_total_antepenultimo,anyo_ingreso,preferencia_seleccion,grupos_por_tipocredito_hash,n_resource_days_m1,pft_events_m7,cohort_year,target
0,100.0,NaN,NaN,NaN,NaN,NaN,5.5,89.29,NaN,0.0,NaN,NaN,NAP,NaN,NaN,207.66,37.5,NaN,NaN,85.71,NaN,NaN,NaN,0.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,31.5,NaN,NaN,NaN,32.34,37.5,378cc790fd99,NaN,NaN,620c9c332101,L,NaN,0.0,NaN,NaN,NaN,42.0,0.0,NaN,NaN,NaN,TC,0.0,NaN,55.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,37.5,NaN,NaN,NaN,NaN,A,100.0,4.0,NaN,NaN,0,NaN,0.0,NaN,10.5,NaN,60.0,0.0,NaN,NaN,0,4.0,2023-06-28 14:19:30,0.0,NaN,0.0,10.5,2012,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,37.5,NaN,NaN,NaN,27.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,42.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,100.0,42.0,NaN,60.0,319636fc9270,NaN,e4f95d56d90df35e,NaN,NaN,NaN,NaN,F,NaN,NaN,1.0,3.0,3.66,0.0,NaN,2021,9.456,NaN,NaN,NaN,0,NaN,NaN,NaN,100.0,2012.0,NaN,e7d0293bc1c5cb4a,NaN,NaN,2021,0
1,100.0,NaN,NaN,NaN,NaN,NaN,6.0,89.29,NaN,0.0,NaN,NaN,NAP,NaN,NaN,207.66,37.5,NaN,NaN,85.71,NaN,NaN,NaN,0.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,31.5,NaN,NaN,NaN,32.34,37.5,b2677d6af4ae,NaN,NaN,620c9c332101,L,NaN,0.0,NaN,NaN,NaN,42.0,0.0,NaN,NaN,NaN,TC,0.0,NaN,55.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,37.5,NaN,NaN,NaN,NaN,A,100.0,4.

We need to check how much data is actually missing or can be replaced with values and used

Some high cardinality values can be reduced to data range like the "pft_minutes"
Low cardinality (<=1) should be dropped (don't ignore NaN for those columns)

In [28]:
# Identify high cardinality columns (many unique values)
print(df_combined.nunique(dropna=False).sort_values(ascending=False))
# 0 variance column
df_combined = df_combined.drop(columns=['ajuste1'])
# Drop identifiers and constants based on study descriptions
ids_to_drop = [
    'dni_hash',                      # Student ID (27k unique)
    'grupos_por_tipocredito_hash',   # Group ID (8k unique - too granular)
    'fecha_datos',                   # Data fetch timestamp
    'caca',                          # Academic year of the data
]
df_combined = df_combined.drop(columns=ids_to_drop)
print(f"New shape after dropping IDs and constants: {df_combined.shape}")

pft_total_minutes_m6              131079
pft_total_minutes_m1              128712
pft_total_minutes_m5              126871
pft_total_minutes_m2              120068
pft_total_minutes_m3              117964
pft_total_minutes_m4              112150
pft_total_minutes_m7              105874
pft_total_minutes_m9              104492
pft_total_minutes_m8               98173
pft_total_minutes_m10              95493
dni_hash                           27067
pft_total_minutes_m11              17319
grupos_por_tipocredito_hash         8351
nota14_hash                         6081
asi_hash                            4249
nota10_hash                         3885
cred_sup_tit                        1627
cred_pend_sup_tit                   1262
rendimiento_total                   1006
pft_events_m9                        995
pft_events_m3                        958
pft_events_m7                        922
pft_events_m5                        890
rend_total_ultimo                    861
pft_events_m10  

Fix the NaNs by replacing them with 0's

Based on study descriptions

In [29]:
# 1. List the specific binary columns (Where NaN = "No")
binary_cols = ['impagado_curso_mat', 'es_adaptado', 'es_retitulado', 'exento_npp']
# 2. Find all activity columns dynamically (Where NaN = "Zero activity")
# This grabs every column starting with 'pft_' or 'resource_'
activity_cols = [col for col in df_combined.columns 
                 if col.startswith('pft_') or col.startswith('resource_')]

cols_to_fix = binary_cols + activity_cols

# 4. Impute with 0
df_combined[cols_to_fix] = df_combined[cols_to_fix].fillna(0)

# 5. Ensure they are numeric (floats/ints) instead of objects
# This prevents XGBoost from crashing on "string" versions of numbers
for col in cols_to_fix:
    df_combined[col] = pd.to_numeric(df_combined[col])

print(f"Successfully imputed {len(cols_to_fix)} columns with 0.")

Successfully imputed 81 columns with 0.


In [ ]:
missing_series = df_combined.isnull().mean()
high_nan_cols = missing_series[missing_series > 0.5].index.tolist()
results = []

print(f"Analyzing {len(high_nan_cols)} high-NaN columns...\n")

for col in high_nan_cols:
    # Skip the target itself or ID columns
    if col in ['abandono_hash', 'dni_hash', 'target']:
        continue
    
    # METRIC A: Does being 'missing' predict dropout?
    # We correlate the boolean "Is Missing" with the Target
    missing_correlation = df[col].isnull().astype(int).corr(df['target'])
    
    # METRIC B: If we fill with 0, does the value predict dropout?
    # (Only works for numeric columns)
    if pd.api.types.is_numeric_dtype(df[col]):
        filled_correlation = df[col].fillna(0).corr(df['target'])
    else:
        filled_correlation = np.nan

    results.append({
        'feature': col,
        'missing_pct': missing_series[col],
        'missingness_corr': missing_correlation, # Correlation of "Is NaN" vs Target
        'zero_filled_corr': filled_correlation   # Correlation of "Value=0" vs Target
    })

# 4. Create a DataFrame to view the strongest signals
results_df = pd.DataFrame(results)

# Sort by absolute correlation of missingness
results_df['signal_strength'] = results_df['missingness_corr'].abs()
top_predictors = results_df.sort_values('signal_strength', ascending=False).head(20)

print(top_predictors[['feature', 'missing_pct', 'missingness_corr', 'zero_filled_corr']])


Analyzing 101 high-NaN columns...



XGBoost handles missing data well, but having more than 40% of column empty is useless

Either replace NaN with value for useful features like 
Or drop

XGboost is good at adapting to missing values, but having more than ~30% of column empty is not useful
Drop anything above that threshhold

In [ ]:
cols_to_drop = missing_percent[missing_percent > 30].index.tolist()

print(f"Dropping {len(cols_to_drop)} columns: {cols_to_drop}")

df_cleaned = df_combined.drop(columns=cols_to_drop)

print(f"New shape: {df_cleaned.shape}")
df_cleaned.head()